In [ ]:
!rm -rf RAW

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import seaborn as sns

from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder.appName("DataExploration").getOrCreate()

pd.options.display.max_columns = None
pd.options.display.max_rows = 250
pd.options.display.max_colwidth = 150
sns.set(color_codes=True)

schema = "polarity FLOAT, id LONG, date_time STRING, query STRING, user STRING, text STRING"

IN_PATH_1 = "/content/drive/MyDrive/sentiment-140-training-data/testdata.manual.2009.06.14.csv"
IN_PATH_2 = "/content/drive/MyDrive/sentiment-140-training-data/training.1600000.processed.noemoticon.csv"
OUT_PATH = "/content/drive/MyDrive/sentiment-140-training-data/RAW"

spark_reader = spark.read.schema(schema)

In [ ]:
test_data = spark_reader.csv(IN_PATH_1, header=False).cache()
training_data = spark_reader.csv(IN_PATH_2, header=False).cache()


In [ ]:
test_data.count()

498

In [ ]:
training_data.count()

1600000

In [ ]:
df = training_data.select("polarity").na.drop()

In [ ]:
#sns.distplot(df.toPandas())

In [ ]:
training_data.groupBy("polarity").count().toPandas()

,polarity,count
0,4.0,800000
1,0.0,800000


In [ ]:
training_data.select("text").na.drop().count()

1600000

In [ ]:
training_data.repartition(20).write.partitionBy("polarity").csv(OUT_PATH, mode="overwrite")

In [ ]:
spark.stop()